## DAtaset

Veamos cómo construir un dataset de lo que ya tenemos

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article, Batch
from django.contrib.auth.models import User

Primero, veamos qué usuarios etiquetaron más de 4 artículos...

In [8]:
from django.db.models import Count
from api.metrics import AgreementCalculator

Article.objects.count()

annotators = sorted([
    "asilva",
    "dvalado",
    "fdesanctis",
    "gclerici",
    "gdamill",
    "lprats",
])

anonymized_names = {
    name:f"annot_{i+1}" for i, name in enumerate(annotators)
}

In [9]:
from django.db.models import Count, Case, When, Value

annotated_articles = Article.objects.exclude(batch__name="training").annotate(
    num_labels=Count('labels'),
    effective_labels=Count(
        Case(When(labels__is_interesting=True, then=Value(1)))
    )
)


really_annotated_articles = annotated_articles.filter(effective_labels=3).count()

skipped = 0

for art in annotated_articles:
    if art.num_labels > art.effective_labels:
        skipped +=1


print(f"Artículos totales: {Article.objects.count()}")
print(f"Artículos con 3 anotaciones: {really_annotated_articles}")
print(f"Artículos con alguna anotación: {annotated_articles.filter(num_labels__gte=1).count()}")
print(f"Artículos que no pasaron a tercera anotación: {skipped}")

Artículos totales: 1108
Artículos con 3 anotaciones: 408
Artículos con alguna anotación: 719
Artículos que no pasaron a tercera anotación: 180


Vamos a crear primero tabla de comentarios

In [10]:
import copy

# Ignoramos OTROS
translation = {
    "MUJER": "WOMEN",
    "LGBTI": "LGBTI",
    "RACISMO": "RACISM",
    "POBREZA": "CLASS",
    "POLITICA": "POLITICS",
    "DISCAPACIDAD": "DISABLED",
    "ASPECTO": "APPEARANCE",
    "CRIMINAL": "CRIMINAL"
}

field_mapping = {
    "HATE": "is_hateful",
    "CALLS": "calls_for_action",
}

for name_spanish, name_english in translation.items():
    field_mapping[name_english] = CommentLabel.type_mapping[name_spanish]

field_mapping

{'HATE': 'is_hateful',
 'CALLS': 'calls_for_action',
 'WOMEN': 'against_women',
 'LGBTI': 'against_lgbti',
 'RACISM': 'against_race',
 'CLASS': 'against_poor',
 'POLITICS': 'against_political',
 'DISABLED': 'against_disabled',
 'APPEARANCE': 'against_aspect',
 'CRIMINAL': 'against_criminals'}

In [18]:
comments = []

def serialize(article, comment, anonymize=True, anonymize_user=True):
    """
    Serialize comment
    """
    ret = {
        **{
            "id": comment.id,
            "text": comment.text,
            "annotators": [],
        },
        **{key:[] for key in field_mapping}
    }
    
    for label in comment.labels.prefetch_related('article_label', 'article_label__user').all():
        username = label.article_label.user.username
        
        if anonymize or anonymize_user:
            annotator_name = anonymized_names[username]
        else:
            annotator_name = username
        
        ret["annotators"].append(annotator_name)
        for name, field in field_mapping.items():
            # Si es verdadero, agregamos
            if getattr(label, field):
                ret[name].append(annotator_name)
                
    return ret


finished_articles = annotated_articles.filter(effective_labels=3).prefetch_related('comment_set', 'labels')

for article in tqdm(finished_articles):
    for comment in article.comment_set.all():
        comments.append(serialize(article, comment))

  0%|          | 0/408 [00:00<?, ?it/s]

In [17]:
def print_comment(comment):
    print(comment['text'])

    for name in field_mapping:
        if comment[name]:
            votes = len(comment[name])
            print(f"{name} --> {votes} votos ({' - '.join(comment[name])})")


for comment in comments:
    if len(comment["HATE"]):
        print("="*80)
        print_comment(comment)
        


@usuario vienen medicos cubanos... ahhh pero no veo ninguna ayuda norteamericana, quienes ni siquiera les importa, seguro que ellos generaron esto, lacras de mierda
HATE --> 1 votos (annot_2)
RACISM --> 1 votos (annot_2)
@usuario No se cayó el avion?? Lpm
HATE --> 1 votos (annot_2)
POLITICS --> 1 votos (annot_2)
@usuario ¡Que cirujeada esta la supuesta enfermita!  Hipocritas.
HATE --> 3 votos (annot_5 - annot_2 - annot_1)
WOMEN --> 2 votos (annot_5 - annot_1)
APPEARANCE --> 1 votos (annot_2)
@usuario Presas tienen q estar está dos mierdas...presas
HATE --> 1 votos (annot_2)
CALLS --> 1 votos (annot_2)
POLITICS --> 1 votos (annot_2)
@usuario Tendrían q ir directamente a la cárcel de Ezeiza. Por Truchas, chorras y fieras! 🤮
HATE --> 3 votos (annot_5 - annot_2 - annot_1)
CALLS --> 1 votos (annot_2)
WOMEN --> 1 votos (annot_5)
POLITICS --> 1 votos (annot_2)
APPEARANCE --> 1 votos (annot_1)
@usuario ¿Y cuando las aislan por chorras?
HATE --> 1 votos (annot_2)
POLITICS --> 1 votos (annot_2)
